# Preparing Spotify Data for Power BI

In [1]:
import pandas as pd
import numpy as np
import requests

In [3]:
# read your 1+ StreamingHistory files (depending on how extensive your streaming history is) into pandas dataframes
df_stream = pd.read_json('StreamingHistory0.json')

# merge streaming dataframes if there is more than one json file
# df_stream = pd.concat([df_stream0, df_stream1])

# create a 'UniqueID' for each song by combining the fields 'artistName' and 'trackName'
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']

df_stream.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2020-06-04 08:56,Major Lazer,Cold Water (feat. Justin Bieber & MØ),18820,Major Lazer:Cold Water (feat. Justin Bieber & MØ)
1,2020-10-16 05:35,Tiësto,Acordeão,131204,Tiësto:Acordeão
2,2020-10-16 05:35,Burak Yeter,My Life Is Going On - Burak Yeter Remix,4626,Burak Yeter:My Life Is Going On - Burak Yeter ...
3,2020-10-16 05:35,Parx,Fantasy,1570,Parx:Fantasy
4,2020-10-16 05:35,Gorgon City,Saving My Life,1168,Gorgon City:Saving My Life


In [10]:
# read your edited Library json file into a pandas dataframe
df_library = pd.read_json('YourLibrary.json')

In [11]:
# add UniqueID column (same as above)
df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']

# add column with track URI stripped of 'spotify:track:'
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]

df_library.head()

,artist,album,track,uri,UniqueID,track_uri
0,Paul Kalkbrenner,Berlin Calling (The Soundtrack by Paul Kalkbre...,QSA,spotify:track:3khntzHFE5TrhT7OKrFlPh,Paul Kalkbrenner:QSA,3khntzHFE5TrhT7OKrFlPh
1,One Direction,Running Hits 2015.04,Drag Me Down,spotify:track:3AtYBAALKWcqxZs0KEKt4r,One Direction:Drag Me Down,3AtYBAALKWcqxZs0KEKt4r
2,Miuosh,MIUOSH | SMOLIK | NOSPR,Perseidy,spotify:track:24kXwnMOucjqg4GAssQLks,Miuosh:Perseidy,24kXwnMOucjqg4GAssQLks
3,The Six,Running Hits 2015.04,Unfinished Sympathy (feat. Jasmine Thompson),spotify:track:2q8RkxBduttply1L2qF0rJ,The Six:Unfinished Sympathy (feat. Jasmine Tho...,2q8RkxBduttply1L2qF0rJ
4,Armin van Buuren,Running Hits 2015.04,Another You (feat. Mr. Probz) - Radio Edit,spotify:track:57PlSDssRq1nXDqxNnfp7R,Armin van Buuren:Another You (feat. Mr. Probz)...,57PlSDssRq1nXDqxNnfp7R


In [12]:
# create final dict as a copy df_stream
df_powerbi = df_stream.copy()

# add column checking if streamed song is in library
# not used in this project but could be helpful for cool visualizations
df_powerbi['In Library'] = np.where(df_powerbi['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)

# left join with df_library on UniqueID to bring in album and track_uri
df_powerbi = pd.merge(df_powerbi, df_library[['album','UniqueID','track_uri']],how='left',on=['UniqueID'])

df_powerbi.head()

,endTime,artistName,trackName,msPlayed,UniqueID,In Library,album,track_uri
0,2020-06-04 08:56,Major Lazer,Cold Water (feat. Justin Bieber & MØ),18820,Major Lazer:Cold Water (feat. Justin Bieber & MØ),0,NaN,NaN
1,2020-10-16 05:35,Tiësto,Acordeão,131204,Tiësto:Acordeão,0,NaN,NaN
2,2020-10-16 05:35,Burak Yeter,My Life Is Going On - Burak Yeter Remix,4626,Burak Yeter:My Life Is Going On - Burak Yeter ...,0,NaN,NaN
3,2020-10-16 05:35,Parx,Fantasy,1570,Parx:Fantasy,0,NaN,NaN
4,2020-10-16 05:35,Gorgon City,Saving My Life,1168,Gorgon City:Saving My Life,0,NaN,NaN


# Creating Genre Dataframe

In [14]:
# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = 'client-id'
CLIENT_SECRET = 'client-secret'

In [15]:
# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [16]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [17]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [18]:
# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()

# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

In [19]:
# convert dictionary into dataframe with track_uri as the first column
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre.head()

,track_uri,artist_uri,genres
0,3khntzHFE5TrhT7OKrFlPh,0rasA5Z5h1ITtHelCpfu9R,"[electronica, german techno, leipzig electroni..."
1,3AtYBAALKWcqxZs0KEKt4r,4AK6F7OLvEQ5QYCBNiQWHq,"[boy band, pop, post-teen pop, talent show]"
2,24kXwnMOucjqg4GAssQLks,3BroLrMp9Q01yOnhLn9qxX,"[polish alternative rap, polish hip hop, polis..."
3,2q8RkxBduttply1L2qF0rJ,5vUOpaF96QTRjj0o1ycWtz,[]
4,57PlSDssRq1nXDqxNnfp7R,0SfsnGyD8FpIN4U4WCkBZ5,"[dutch trance, edm, pop dance, progressive hou..."


In [20]:
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

,track_uri,artist_uri,genres
0,3khntzHFE5TrhT7OKrFlPh,0rasA5Z5h1ITtHelCpfu9R,electronica
0,3khntzHFE5TrhT7OKrFlPh,0rasA5Z5h1ITtHelCpfu9R,german techno
0,3khntzHFE5TrhT7OKrFlPh,0rasA5Z5h1ITtHelCpfu9R,leipzig electronic
0,3khntzHFE5TrhT7OKrFlPh,0rasA5Z5h1ITtHelCpfu9R,minimal techno
0,3khntzHFE5TrhT7OKrFlPh,0rasA5Z5h1ITtHelCpfu9R,tech house


In [21]:
# save df_tableau and df_genre_expanded as csv files that we can load into Tableau
df_powerbi.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpandedTable.csv')